Using DeepChem with Tensorflow Data and Estimators
-----------------------------------------------

When DeepChem was first created, Tensorflow had no standard interface for datasets or models.  We created the Dataset and Model classes to fill this hole.  More recently, Tensorflow has added the `tf.data` module as a standard interface for datasets, and the `tf.estimator` module as a standard interface for models.  To enable easy interoperability with other tools, we have added features to Dataset and Model to support these new standards.

This example demonstrates how to use these features.  Let's begin by loading a dataset and creating a model to analyze it.  We'll use a simple MultiTaskClassifier with one hidden layer.

In [1]:
import deepchem as dc
import tensorflow as tf
import numpy as np

tasks, datasets, transformers = dc.molnet.load_tox21()
train_dataset, valid_dataset, test_dataset = datasets
n_tasks = len(tasks)
n_features = train_dataset.X.shape[1]

model = dc.models.MultiTaskClassifier(n_tasks, n_features, layer_sizes=[1000], dropouts=0.25)

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


We want to train the model using the training set, then evaluate it on the test set.  As our evaluation metric we will use the ROC AUC, averaged over the 12 tasks included in the dataset.  First let's see how to do this with the DeepChem API.

In [2]:
model.fit(train_dataset, nb_epoch=100)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
print(model.evaluate(test_dataset, [metric]))

computed_metrics: [0.7081719239992621, 0.865129545326814, 0.836842947009276, 0.7701224617961064, 0.7081635765485087, 0.7911911911911912, 0.7207671300893743, 0.6592307518932563, 0.7976869777868352, 0.7409154581410679, 0.8243317675424011, 0.7112435328898743]
{'mean-roc_auc_score': 0.7611497720178306}


Simple enough.  Now let's see how to do the same thing with the Tensorflow APIs.  Fair warning: this is going to take a lot more code!

To begin with, Tensorflow doesn't allow a dataset to be passed directly to a model.  Instead, you need to write an "input function" to construct a particular set of tensors and return them in a particular format.  Fortunately, Dataset's `make_iterator()` method provides exactly the tensors we need in the form of a `tf.data.Iterator`.  This allows our input function to be very simple.

In [3]:
def input_fn(dataset, epochs):
    x, y, weights = dataset.make_iterator(batch_size=100, epochs=epochs).get_next()
    return {'x': x, 'weights': weights}, y

Next, you have to use the functions in the `tf.feature_column` module to create an object representing each feature and weight column (but curiously, *not* the label column—don't ask me why!).  These objects describe the data type and shape of each column, and give each one a name.  The names must match the keys in the dict returned by the input function.

In [4]:
x_col = tf.feature_column.numeric_column('x', shape=(n_features,))
weight_col = tf.feature_column.numeric_column('weights', shape=(n_tasks,))

Unlike DeepChem models, which allow arbitrary metrics to be passed to `evaluate()`, estimators require all metrics to be defined up front when you create the estimator.  Unfortunately, Tensorflow doesn't have very good support for multitask models.  It provides an AUC metric, but no easy way to average this metric over tasks.  We therefore must create a separate metric for every task, then define our own metric function to compute the average of them.

In [5]:
def mean_auc(labels, predictions, weights):
    metric_ops = []
    update_ops = []
    for i in range(n_tasks):
        metric, update = tf.metrics.auc(labels[:,i], predictions[:,i], weights[:,i])
        metric_ops.append(metric)
        update_ops.append(update)
    mean_metric = tf.reduce_mean(tf.stack(metric_ops))
    update_all = tf.group(*update_ops)
    return mean_metric, update_all

Now we create our `Estimator` by calling `make_estimator()` on the DeepChem model.  We provide as arguments the objects created above to represent the feature and weight columns, as well as our metric function.

In [6]:
estimator = model.make_estimator(feature_columns=[x_col],
                                 weight_column=weight_col,
                                 metrics={'mean_auc': mean_auc},
                                 model_dir='estimator')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12d39bef0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


We are finally ready to train and evaluate it!  Notice how the input function passed to each method is actually a lambda.  This allows us to write a single function, then use it with different datasets and numbers of epochs.

In [7]:
estimator.train(input_fn=lambda: input_fn(train_dataset, 100))
print(estimator.evaluate(input_fn=lambda: input_fn(test_dataset, 1)))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into estimator/model.ckpt.
INFO:tensorflow:loss = 1716.8385, step = 1
INFO:tensorflow:global_step/sec: 63.5117
INFO:tensorflow:loss = 804.13416, step = 101 (1.576 sec)
INFO:tensorflow:global_step/sec: 41.9688
INFO:tensorflow:loss = 682.90265, step = 201 (2.383 sec)
INFO:tensorflow:global_step/sec: 65.7841
INFO:tensorflow:loss = 805.00336, step = 301 (1.520 sec)
INFO:tensorflow:global_step/sec: 42.4989
INFO:tensorflow:loss = 466.05975, step = 401 (2.353 sec)
INFO:tensorflow:global_step/sec: 65.1989
INFO:tensorflow:loss = 704.69446, step = 501 (1.534 sec)
INFO:tensorflow:global_step/sec: 40.9395
INFO:tensorflow:loss = 450.0899, step = 601 (2.443 sec)
INFO:tensorflow:global_step/sec: 41.2419
INFO:tensorflow:loss = 349.05804, step = 701 (2.424 sec)
INFO:tensorflow:global_step/sec: 60.7104
INFO:tensorflow:loss = 338.63837, step = 801 (1.647 sec)
INFO:tensorflow:global_step/sec: 40.4742
INFO:tensorflow:loss 

That's a lot of code for something DeepChem can do in three lines.  The Tensorflow API is verbose and somewhat confusing.  It has seemingly arbitrary limitations, like assuming a model will only ever have one output, and therefore only allowing one label.  But for better or worse, it's a standard.

Of course, if you just want to use a DeepChem model with a DeepChem dataset, there is no need for any of this.  Just use the DeepChem API.  But perhaps you want to use a DeepChem dataset with a model that has been implemented as an estimator.  In that case, `Dataset.make_iterator()` allows you to easily do that.  Or perhaps you have higher level workflow code that is written to work with estimators.  In that case, `make_estimator()` allows DeepChem models to easily fit into that workflow.